Loading our JSON Data


In [16]:
import nltk
from nltk.stem.lancaster import LancasterStemmer
stemmer = LancasterStemmer()

import numpy
import tflearn
import tensorflow as tf
import random

import json
with open('intents.json') as file:
    data = json.load(file)

Extracting Data
- setup some blank lists to store these values.

In [17]:
words = []
labels = []
docs_x = []
docs_y = []

 loop through our JSON data and extract the data we want. 
 For each pattern we will turn it into a list of words using nltk.word_tokenizer, rather than having them as strings.
add each pattern into  docs_x list and its associated tag into the docs_y list.

In [18]:
for intent in data['intents']:
    for pattern in intent['patterns']:
        wrds = nltk.word_tokenize(pattern)
        words.extend(wrds)
        docs_x.append(wrds)
        docs_y.append(intent["tag"])
        
    if intent['tag'] not in labels:
        labels.append(intent['tag'])

Word Stemming

In [19]:
words = [stemmer.stem(w.lower()) for w in words if w != "?"]
words = sorted(list(set(words)))

labels = sorted(labels)

Bag of Words

 If the position in the list is a 1 then that will mean that the word exists in our sentence, if it is a 0 then the word is not present.

In [20]:
training = []
output = []

out_empty = [0 for _ in range(len(labels))]

for x, doc in enumerate(docs_x):
    bag = []

    wrds = [stemmer.stem(w.lower()) for w in doc]

    for w in words:
        if w in wrds:
            bag.append(1)
        else:
            bag.append(0)

    output_row = out_empty[:]
    output_row[labels.index(docs_y[x])] = 1

    training.append(bag)
    output.append(output_row)

convert our training data and output to numpy arrays.

In [21]:
training = numpy.array(training)
output = numpy.array(output)

Developing a Model
-fairly standard feed-forward neural network with two hidden layers.
-give a class that they belong too

In [24]:
tf.compat.v1.reset_default_graph()

net = tflearn.input_data(shape=[None, len(training[0])])
net = tflearn.fully_connected(net, 8)
net = tflearn.fully_connected(net, 8)
net = tflearn.fully_connected(net, len(output[0]), activation="softmax")
net = tflearn.regression(net)

model = tflearn.DNN(net)

Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor


Training & Saving the Model

In [25]:
model.fit(training, output, n_epoch=1000, batch_size=8, show_metric=True)
model.save("model.tflearn")

Training Step: 2999  | total loss: 0.20864 | time: 0.006s
| Adam | epoch: 1000 | loss: 0.20864 - acc: 0.9749 -- iter: 16/20
Training Step: 3000  | total loss: 0.17951 | time: 0.009s
| Adam | epoch: 1000 | loss: 0.17951 - acc: 0.9774 -- iter: 20/20
--
INFO:tensorflow:c:\Users\Shemm\Desktop\Lastminute\model.tflearn is not in all_model_checkpoint_paths. Manually adding it.


In [26]:
def bag_of_words(s, words):
    bag = [0 for _ in range(len(words))]

    s_words = nltk.word_tokenize(s)
    s_words = [stemmer.stem(word.lower()) for word in s_words]

    for se in s_words:
        for i, w in enumerate(words):
            if w == se:
                bag[i] = 1
            
    return numpy.array(bag)


def chat():
    print("Start talking with the bot (type quit to stop)!")
    while True:
        inp = input("You: ")
        if inp.lower() == "quit":
            break

        results = model.predict([bag_of_words(inp, words)])
        results_index = numpy.argmax(results)
        tag = labels[results_index]

        for tg in data["intents"]:
            if tg['tag'] == tag:
                responses = tg['responses']

        print(random.choice(responses))

chat()

Start talking with the bot (type quit to stop)!
Good to see you again
